# **Classification**

### Import des librairies

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.preprocessing import normalize
import numpy as np
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### Chargemet des données Iris

In [ ]:
dataset = load_iris()

X = dataset['data']
Y = dataset['target']

attributs = dataset['feature_names']

print('Liste des attributs:',attributs)

print('Nombre d\'instances est:',X.shape[0])
print('Nombre d\'attributs est:',X.shape[1])

print("Liste de ce que l'on veut prédire :",np.unique(Y))

### Transformer le problème en classification binaire `[0,1]`

In [ ]:
# on crée une copie de Y pour la transformer en binaire
Y_binaire = Y.copy()
# on met ce qui n'est pas 0 (Setosa) à 1
Y_binaire[Y!=0] = 1

print("Liste de ce que l'on veut prédire:", np.unique(Y_binaire))

### Définition d'une fonction qui transforme en représentation binaire pour plusieurs classes

In [ ]:
def transform_labels(y_train,y_test):
  """
  Cette fonction transforme les classes non-binaires en une représentation binaire
  Par exemple si on a une liste de 6 fleurs chacune peut avoir une des 3 classes
  Entrée: [
           1,
           3,
           3,
           2,
           1,
           2
          ]

  Sortie: [
           [1,0,0], # class 1
           [0,0,1], # class 3
           [0,0,1], # class 3
           [0,1,0], # class 2
           [1,0,0], # class 1
           [0,1,0]  # class 2
          ]
  """

  print('y_train',y_train.shape)
  print('y_test',y_test.shape)

  # concatener train et test
  y_train_test = np.concatenate((y_train,y_test),axis =0)

  # init un encoder Label
  encoder = LabelEncoder()
  # transformer de [1,3,3,2,1,2] à [0,2,2,1,0,1]
  new_y_train_test = encoder.fit_transform(y_train_test)

  # init un encoder one-hot
  encoder = OneHotEncoder()
  # transformer de [0,2,2,1,0,1] à la représentation binaire
  new_y_train_test = encoder.fit_transform(new_y_train_test.reshape(-1,1))

  # resplit the train and test
  new_y_train = new_y_train_test[0:len(y_train)]
  new_y_test = new_y_train_test[len(y_train):]

  print('new_y_train',new_y_train.shape)
  print('new_y_test',new_y_test.shape)

  return new_y_train, new_y_test

### Diviser en données d'entrainement et données de test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,Y_binaire,test_size= 33/100)

print('Nombre d\'instances pour l\'entrainement:',X_train.shape[0])
print('Nombre d\'instances pour le test:',X_test.shape[0])

### Normalizer `X_train` et `X_test` pour que toutes les caractéristiques soient entre 0 et 1

In [ ]:
X_train = normalize(X_train)
X_test = normalize(X_test)

### Choisir les attributs sur lequels on veut travailler

In [ ]:
index_des_attributs = [0,1]

x_train = X_train[:,index_des_attributs]
x_test = X_test[:,index_des_attributs]

## Créer un modèle de régression logistique avec Keras

### Créer la couche d'entrée qui a la même shape que celle d'une instance dans `x_train`

In [ ]:
input_shape = x_train.shape[1:]

input_layer = keras.layers.Input(input_shape)

### Créer une couche qui contient un neurone qui prend en entrée l'`input_layer` avec une activation 'sigmoid' (sigma)

In [ ]:
output_layer = keras.layers.Dense(units=1, activation='sigmoid')(input_layer)

### Création du modèle (qui est un perceptron)

In [ ]:
model = keras.models.Model(inputs=input_layer, outputs=output_layer)

model.summary()

### Choisir l'algorithme d'optimisation avec un learning rate de 0.1

In [ ]:
learning_rate = 0.1
optimizer_algo = keras.optimizers.SGD(learning_rate=learning_rate)

### Choisir la fonction de coût qu'on veut optimiser : *Binary Cross-Entropy*

In [ ]:
cost_function = keras.losses.binary_crossentropy

### Compiler le modèle en lui indiquant qu'on veut aussi mesurer l'accuracy

In [ ]:
model.compile(loss=cost_function,optimizer=optimizer_algo, metrics=['accuracy'])

### Entrainer le modèle avec un batch size de 32 pour 1000 époques

In [ ]:
mini_batch_size = 16
nb_epochs = 1000
history = model.fit(x_train,y_train,batch_size=mini_batch_size,epochs=nb_epochs,verbose=False)

### Tracer la variation du taux d'erreur sur le train en fonction du nombre d'epoque

In [ ]:
history_dict = history.history
loss_epochs = history_dict['loss']

plt.figure()
plt.plot(loss_epochs)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()
plt.close()

### Évaluer ce modèle sur l'ensemble du train et test

In [ ]:
loss,acc = model.evaluate(x_train,y_train,verbose=False)

print("L'accuracy sur l'ensemble du train est:",acc)

loss,acc = model.evaluate(x_test,y_test,verbose=False)

print("L'accuracy sur l'ensemble du test est:",acc)

## Tracer et visualiser les données dans un cas  2D

### Tracer les données sans la décision

In [ ]:
if len(index_des_attributs) == 2:
  colors = np.empty(y_train.shape,dtype=str)
  # rouge pour la première classe (Setosa)
  colors[y_train==0] = 'red'
  # bleu pour la deuxième classe (non-Setosa)
  colors[y_train==1] = 'blue'
  # tracer
  plt.figure()
  plt.ylabel('x_1')
  plt.xlabel('x_2')
  plt.scatter(x_train[:,0],x_train[:,1],c=colors)
  plt.savefig('plot-2d-iris.pdf')
  plt.show()
  plt.close()

### Tracer les données avec la décision

In [ ]:
if len(index_des_attributs) == 2:
  colors = np.empty(y_train.shape,dtype=str)
  # rouge pour la première classe (Setosa)
  colors[y_train==0] = 'red'
  # bleu pour la deuxième classe (non-Setosa)
  colors[y_train==1] = 'blue'

  # chercher les minimums et maximum des attributs
  minimum_x_1 = x_train[:,0].min()-0.05
  minimum_x_2 = x_train[:,1].min()-0.05
  maximum_x_1 = x_train[:,0].max()+0.05
  maximum_x_2 = x_train[:,1].max()+0.05

  # pour remplir l'espace avec les décisions
  # spécifier la précision des points de la surface de décision
  step = 0.001

  # créer le plan pour lui prédire la décision du classifieur
  x_1,x_2 = np.mgrid[minimum_x_1:maximum_x_1:step,minimum_x_2:maximum_x_2:step]
  grid = np.c_[x_1.ravel(), x_2.ravel()]

  # créer x pour l'entrée du classifieur
  x = grid

  # chercher les probabilités prédites
  y_pred = model.predict(x)

  # maintenant transformer les probabilités en labels prédits
  # ce qui est supérieur à 0.5 est considéré 1 sinon 0
  y_pred[y_pred>0.5] = 1
  y_pred[y_pred<=0.5] = 0

  # changer la forme de la prédiction pour qu'elle corresponde à celle d'un plan
  z = y_pred[:,0].reshape(x_1.shape)

  # créer la figure vide
  f, ax = plt.subplots(figsize=(8, 6))

  # déssiner le contour de la prédiction
  contour = ax.contourf(x_1, x_2, z, 25, cmap="RdBu",
                      vmin=0, vmax=1)
  # mettre la barre de couleur
  ax_c = f.colorbar(contour)

  # tracer les points
  ax.scatter(x_train[:,0],x_train[:,1],c=colors,
            edgecolor="white", linewidth=1)

  # annoter
  plt.ylabel('x_1')
  plt.xlabel('x_2')
  ax_c.set_label("$\hat{y}$")

  # sauvegarder et afficher la figure
  plt.savefig('decision-binary.pdf')
  plt.show()
  plt.close()

In [ ]:
print(y_pred.shape)
print(z.shape)
print(x_1.ravel().shape)

## **Exercices**

## Refaire le travail pour la classification des trois classes `[0,1,2]`

In [ ]:
# Votre code ici

 ### Corrigé :

In [ ]:
# restart keras and tensorflow session
keras.backend.clear_session()

# on divise en train et test
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size= 33/100 )

# Ensuite on normalise entre 0 et 1 toutes les valeurs
X_train = normalize(X_train)
X_test = normalize(X_test)

# on choisit la liste des attributs sur lesquelles on veut travailler
index_des_attributs = [0,1]
x_train = X_train[:,index_des_attributs]
x_test = X_test[:,index_des_attributs]

# récupérer la liste unique des classes
y_train_unique = np.unique(y_train)
# récupérer le nombre de classes C
C = len(y_train_unique)

# transformer les classes en représentation binaire (one-hot encoding)
y_train_binaire,y_test_binaire = transform_labels(y_train,y_test)

# maintenant on créé le modèle
# on commence par la couche d'entrée
input_shape = x_train.shape[1:]
input_layer = keras.layers.Input(input_shape)

# ensuite la couche de sortie avec l'activation softmax
# qui contient C neurones (C étant le nombre de classes)
output_layer = keras.layers.Dense(units=C, activation='softmax')(input_layer)

# créer le modèle en spécifiant input et output
model = keras.models.Model(inputs=input_layer, outputs=output_layer)

# choisir le taux d'apprentissage
learning_rate = 0.1

# choisir l'algorithme d'optimsation en lui spécifiant le taux d'aprentissage
optimizer_algo = keras.optimizers.SGD(learning_rate=learning_rate)

# choisir la fonction de coût: categorical cross entropy
cost_function = keras.losses.categorical_crossentropy

# compiler le modèle en lui spécifiant qu'on veut surveiller l'accuracy
model.compile(loss=cost_function,optimizer=optimizer_algo, metrics=['accuracy'])

# afficher les information du modèle
model.summary()

# choisir le batch size
mini_batch_size = 8

# choisir le nombre d'époque
nb_epochs = 1000

# conversion en array numpy (TF2)
y_train_binaire = y_train_binaire.toarray()
y_test_binaire = y_test_binaire.toarray()

# commencer l'entrainement
history = model.fit(x_train,y_train_binaire,batch_size=mini_batch_size,epochs=nb_epochs,verbose=False)

# evaluation sur train
loss,acc = model.evaluate(x_train,y_train_binaire,verbose=False)
print("L'accuracy sur l'ensemble du train est :",acc)

# evaluation sur le test
loss,acc = model.evaluate(x_test,y_test_binaire,verbose=False)
print("L'accuracy sur l'ensemble du test est :",acc)

### Tracer les données dans un plan 2D mais avec trois couleurs (trois classes)

In [ ]:
# Votre code ici

 ### Corrigé :

In [ ]:
if len(index_des_attributs) == 2:
  colors = np.empty(y_train.shape,dtype=str)
  # rouge pour la première classe
  colors[y_train==0] = 'red'
  # bleu pour la deuxième classe
  colors[y_train==1] = 'blue'
  # vert pour la troisème classe
  colors[y_train==2] = 'green'
  # tracer
  plt.figure()
  plt.ylabel('x_1')
  plt.xlabel('x_2')
  plt.scatter(x_train[:,0],x_train[:,1],c=colors)
  plt.savefig('plot-2d-iris.pdf')
  plt.show()
  plt.close()

### Tracer la décision dans le cas de trois classes en utilisant [cmaps](https://matplotlib.org/examples/color/colormaps_reference.html)

In [ ]:
# Votre code ici

 ### Corrigé :

In [ ]:
if len(index_des_attributs) == 2:
  colors = np.empty(y_train.shape,dtype=str)
  # rouge pour la première classe (Setosa)
  colors[y_train==0] = 'blue'
  # bleu pour la deuxième classe (non-Setosa)
  colors[y_train==1] = 'red'
  # vert pour la troisème classe
  colors[y_train==2] = 'green'

  # chercher les minimums et maximum des attributs
  minimum_x_1 = x_train[:,0].min()-0.05
  minimum_x_2 = x_train[:,1].min()-0.05
  maximum_x_1 = x_train[:,0].max()+0.05
  maximum_x_2 = x_train[:,1].max()+0.05

  # pour remplir l'espace avec les décisions
  # spécifier la précision des points de la surface de décision
  step = 0.001

  # créer le plan pour lui prédire la décision du classifieur
  x_1,x_2 = np.mgrid[minimum_x_1:maximum_x_1:step,minimum_x_2:maximum_x_2:step]
  grid = np.c_[x_1.ravel(), x_2.ravel()]

  # créer x pour l'entrée du classifieur
  x = grid

  # chercher les probabilités prédites
  y_pred = model.predict(x)

  print("y_pred avant la transformation",y_pred.shape)

  # maintenant transformer les probabilités en labels prédits
  # en utilisant la fonction numpy.argmax
  y_pred = np.argmax(y_pred,axis=1)

  print("y_pred après la transformation",y_pred.shape)

  # changer la forme de la prédiction pour qu'elle correspon à celle d'un plan
  z = y_pred.reshape(x_1.shape)

  # créer la figure vide
  f, ax = plt.subplots(figsize=(8, 6))

  # déssiner le contour de la prédiction
  contour = ax.contourf(x_1, x_2, z, 25, cmap="brg",
                      vmin=0, vmax=2)
  # mettre la barre de couleure
  ax_c = f.colorbar(contour)

  # tracer les points
  ax.scatter(x_train[:,0],x_train[:,1],c=colors,
            edgecolor="white", linewidth=1)

  # annoter
  plt.ylabel('x_1')
  plt.xlabel('x_2')
  ax_c.set_label("$\hat{y}$")

  # sauveguarder et afficher la figure
  plt.savefig('decision-binary.pdf')
  plt.show()
  plt.close()